This notebooks provides an example of how to create plots displaying the results of the analysis.<br>
As a 4D analysis is made up of of several 2D interpolations (loops on time and depth), we can use the tools of the `pydiva2d` module to produce the figures for the contours, meshes and results.

The main difference with [plot_diva4D_results](./plot_diva4D_results.ipynb) is that here the figures are produced using the [Basemap](http://matplotlib.org/basemap/api/basemap_api.html) toolkit. 

In [26]:
import os
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from importlib import reload

In [27]:
reload(pydiva4d)

<module 'pydiva4d' from '../pydiva4d.py'>

In [8]:
sys.path.insert(0, "../")
import pydiva4d
import pydiva2d

Set the logging levels.

In [15]:
pydiva2d.logger.setLevel(logging.DEBUG)
pydiva4d.logger.setLevel(logging.DEBUG)

Define a directory for the figures:

In [16]:
figdir = './figures/BlackSea/'
if not os.path.exists(figdir):
    os.makedirs(figdir)

Define the directory where Diva will be executed:

In [29]:
divamaindir = '/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1/'
Diva4Ddirs = pydiva4d.Diva4DDirectories(divamaindir)

In [18]:
Diva4Dfiles = pydiva4d.Diva4Dfiles(Diva4Ddirs.diva4d)

## Load the depth levels

In [19]:
contourdepth = pydiva4d.Contourdepth()
contourdepth.read_from(Diva4Dfiles.contourdepth)
contourdepth.depthlist

[30.0, 20.0, 10.0, 0.0]

# Make the plots

First we initialise the Basemap by selecting a projection and an area of interest.<br>
The domain limits are extracted from the parameter file.

In [32]:
Param = pydiva2d.Diva2DParameters()
Param.read_from(os.path.join(Diva4Ddirs.diva4dinput, 'param.par'))

In [33]:
m = Basemap(projection='merc', llcrnrlon=Param.xori, llcrnrlat=Param.yori, urcrnrlon=Param.xend,
            urcrnrlat=Param.yend, resolution='i')

## Loop on the contour

In [36]:
for idepth, depth in enumerate(contourdepth.depthlist):
    
    contourfile = os.path.join(Diva4Ddirs.diva4dparam, 'coast.cont.{0}'.format(str(10001 + idepth)))

    contour2D = pydiva4d.Diva2DContours()
    contour2D.read_from(contourfile)

    fig = plt.figure()
    contour2D.add_to_plot(m, color='k', linewidth=.5)
    plt.title("Contour at depth: {0} m".format(depth))
    m.drawmeridians(np.linspace(Param.xori, Param.xend, 4), labels=[0, 0, 0, 1], linewidth=0.2)
    m.drawparallels(np.linspace(Param.yori, Param.yend, 4), labels=[1, 0, 0, 0], linewidth=0.2)
    # plt.show()
    plt.savefig(os.path.join(figdir, "BlackSea_basemap_contour{0}".format(idepth)))
    plt.close()

/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3260: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3269: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)


## Loop on the mesh

In [39]:
for idepth, depth in enumerate(contourdepth.depthlist):
    # Create the file names
    meshtopofile = os.path.join(Diva4Ddirs.diva4dmesh, "mesh.dat.{0}".format(str(10000 + idepth + 1)))
    meshfile = os.path.join(Diva4Ddirs.diva4dmesh, "meshtopo.{0}".format(str(10000 + idepth + 1)))
    
    # Mesh object
    Mesh = pydiva4d.Diva2DMesh()
    Mesh.read_from(meshfile, meshtopofile)
    
    # Make the plot
    fig = plt.figure()
    ax = plt.subplot(111)
    Mesh.add_to_plot(m, linewidth=0.1, color='k')
    m.drawmeridians(np.linspace(Param.xori, Param.xend, 4), labels=[0, 0, 0, 1], linewidth=0.2)
    m.drawparallels(np.linspace(Param.yori, Param.yend, 4), labels=[1, 0, 0, 0], linewidth=0.2)
    plt.title("Mesh at depth: {0} m".format(depth))
    plt.savefig(os.path.join(figdir, "BlackSea_basemap_mesh{0}".format(idepth)))
    plt.close()

/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3260: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3269: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)


## Loop on the times and depth levels for the results

Load year and month periods:

In [40]:
Monthlist = pydiva4d.Monthlist()
Monthlist.read_from(Diva4Dfiles.monthlist)
Monthlist.monthlist

['0101', '0202', '0303', '']

In [41]:
Yearlist = pydiva4d.Yearlist()
Yearlist.read_from(Diva4Dfiles.yearlist)
Yearlist.yearlist

['19002009']

## Load the variables

In [42]:
Varlist = pydiva4d.Varlist()
Varlist.read_from(Diva4Dfiles.varlist)
Varlist.varlist

['Temperature', 'Salinity']

# Loop on levels, variables, periods...

In [45]:
for variables in Varlist.varlist:
    for yearperiods in Yearlist.yearlist:
        for monthperiods in Monthlist.monthlist:
            for idepth, depthlevels in enumerate(contourdepth.depthlist):
                
                resultfile = "{0}.{1}.{2}.{3}.anl.nc".format(variables, yearperiods,
                                                             monthperiods, str(10000 + idepth + 1))
                
                figname = ''.join((resultfile.replace('.', '_'), '_basemap.png'))
                figtitle = "{0}, Year: {1}, Months: {2}, Depth: {3} m".format(variables, yearperiods,
                                                                                       monthperiods, str(depthlevels))
                
                if os.path.exists(os.path.join(Diva4Ddirs.diva4dfields, resultfile)):
                    Results = pydiva4d.Diva2DResults(os.path.join(Diva4Ddirs.diva4dfields, resultfile))
                    fig = plt.figure()
                    ax = plt.subplot(111)
                    resultplot = Results.add_to_plot(field='analysis', m=m)
                    m.drawmeridians(np.linspace(Param.xori, Param.xend, 4), labels=[0, 0, 0, 1], linewidth=0.2)
                    m.drawparallels(np.linspace(Param.yori, Param.yend, 4), labels=[1, 0, 0, 0], linewidth=0.2)
                    plt.colorbar(resultplot)
                    plt.title(figtitle)
                    plt.savefig(os.path.join(figdir, figname))
                    plt.close()

/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3413: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3422: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)
